# Big B-Router  
Edit out roads that are in front of ALPRs (in a pbf file)  
Follow the ⭐ instructions before runnning each cell in order.  

# 1.  Import libraries and define small functions  

⭐ Needed python packages are:   
  
<b>├── geopandas  
  ├── folium   
  ├── pyrosm   
  └── pyosmium   
  </b>  

Everything else is a dependency.  


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely as shp
from shapely.geometry import Polygon
from pyproj import Geod

import osmium as omi
from osmium import FileProcessor as FProc
from pyrosm import OSM

import warnings
import re

pd.set_option('display.max_columns', None)

geod = Geod(ellps="WGS84")
# Function to generate trapezoids (ALPR "sightline")
def trapezoidshp(startlng, startlat, direction, height, angle, left_bias, base_width) -> Polygon:
    direction-=left_bias
    hypotenuse = height / np.cos(np.radians((angle/2)))
    bearings = np.array([direction+90, direction-90,direction - angle/2, direction + angle/2])
    lng, lat, _ = geod.fwd([startlng]*4, [startlat]*4, bearings, [base_width/2]*2+[hypotenuse]*2)
    return Polygon([(lng[0], lat[0]),(lng[1], lat[1]), (lng[2], lat[2]), (lng[3], lat[3])])

# Function to split "other_tags" into their own columns
def regparse_othertags(row):
    regpat = r'"([^"]+)"=>"([^"]+)"'
    matches = re.findall(regpat, row)
    result = {key: value for key, value in matches}
    return pd.Series(result)

# Truncates decimal places ; used for matching close lat/lng coordinates
def truncate(f, n):
    return np.floor(f * 10 ** n) / 10 ** n


# 2. Load in ALPR data  
⭐ Replace the filenames (filefullarea is not the state file, but the cropped area)  
Modifications like restricting to flock ALPR's only can be made here w/ geopandas/pandas.  

In [ ]:
filefullarea = "./pbf/area-of-interest.osm.pbf"
filesurv = "./pbf/area-of-interest-surveillance.osm.pbf"
fileroad = "./pbf/area-of-interest-roads.osm.pbf"


surveillance = gpd.read_file(filesurv,layer="points")
surveillance.dropna(axis=0, subset="other_tags", ignore_index=True, inplace=True)
surveillance = surveillance.drop("other_tags",axis=1).join(surveillance["other_tags"].apply(regparse_othertags))
alpr = surveillance[surveillance["surveillance:type"]=="ALPR"].reset_index(drop=True)
alpr = alpr.rename(columns={"direction":"camdir","camera:direction":"camdir2"})

# 3. Project ALPR data  
⭐ Adjust the parameters or run w/ defaults  
Default is trapezoidal projection.      
Parameters  
  ├── angle - The angle of vision to project from the camera (adapted from the original triangle projection)  
  ├── visrange - The (meters) distance away from the camera to project to (this becomes the radius for directionless ALPRs)  
  ├── base_width - Trapezoid base (on position of camera) width in meters  
  └── left_bias - The degrees to tilt direction to the left as cameras tend to be mounted to the right of roads  

In [ ]:
# Trapezoid projection
angle = 45
visrange = 30
base_width = 10
left_bias = 0

alprtrap = gpd.GeoDataFrame()
n=0
for _, cam in alpr.iterrows():
    if (pd.isna(cam["camdir"])):
        if not (pd.isna(cam["camdir2"])):
            for direction in list(filter(None,re.sub("[a-zA-Z]","",str(cam["camdir2"])).replace("-",";").replace(",",";").split(";"))):
                shape = trapezoidshp(cam.geometry.x, cam.geometry.y, int(direction), visrange, angle, left_bias, base_width)
                alprtrap = pd.concat([alprtrap, gpd.GeoDataFrame({"camindex":n,"geometry":[shape]}, crs="EPSG:4326")])
        else:
            _, radius_endp, _ = geod.fwd(cam.geometry.x, cam.geometry.y,0,visrange)
            circ = shp.geometry.Point(cam.geometry.x, cam.geometry.y).buffer(radius_endp-cam.geometry.y)
            alprtrap = pd.concat([alprtrap, gpd.GeoDataFrame({"camindex":n,"geometry":[circ]}, crs="EPSG:4326")])
    else:
        for direction in list(filter(None,re.sub("[a-zA-Z]","",str(cam["camdir"])).replace("-",";").replace(",",";").split(";"))):
            shape = trapezoidshp(cam.geometry.x, cam.geometry.y, int(direction), visrange, angle, left_bias, base_width)
            alprtrap = pd.concat([alprtrap, gpd.GeoDataFrame({"camindex":n,"geometry":[shape]}, crs="EPSG:4326")])
    n+=1
alprtrap = alprtrap.reset_index(drop=True)

# 4. Load in roads and get difference from ALPR projection  
If using a different ALPR projection replace the second value in gpd.overlay()  
This cell takes a bit, coffee break?  

In [ ]:
warnings.filterwarnings("ignore")
roads = OSM(fileroad)
roads = roads.get_network("driving+service")
roads["geometry"] = roads.geometry.line_merge()
roads = roads.explode().reset_index(drop=True)
roads["geo2"] = roads.geometry
diffedroads = gpd.overlay(roads, alprtrap, how="difference")
diff = diffedroads[diffedroads.geom_type=="LineString"]
diff = diffedroads[diffedroads.geometry==diffedroads.geo2]

# 5. Determine which geometries to drop/edit  

In [ ]:
todrop = roads.set_index("id").drop(diffedroads.id)
idstodrop = todrop.index
toedit = diffedroads.drop(diff.index)
explodedit = toedit.explode()
# Get rid of id's with short length (<100m)
explodedit = toedit.explode()
explodedit["length"] = explodedit.to_crs("EPSG:3857").geometry.length
b = explodedit.groupby("id")["length"].sum()
shortsplits = b[b<100].index

idstodrop = np.concatenate((idstodrop, shortsplits))
toedit = toedit.set_index("id").drop(shortsplits)
idstoedit = toedit.index

# 6. Drop/Edit geometries and write to PBF file  
⭐ Adjust output filename if desired  

In [ ]:
outputfile = "./bigB.osm.pbf"

fakeid = (2**45)-1
explodict = explodedit.reset_index(drop=True).groupby("id")
editids = set(idstoedit)
dropids = set(idstodrop)

bothfilter = omi.filter.IdFilter(editids|dropids)
roadsfilter = omi.filter.KeyFilter("highway")

with omi.SimpleWriter(outputfile, overwrite=True) as writer:
    ways = FProc(filefullarea, omi.osm.WAY|omi.osm.NODE)\
            .with_locations()\
            .with_filter(roadsfilter)\
            .with_filter(bothfilter)\
            .with_filter(omi.filter.GeoInterfaceFilter())\
            .handler_for_filtered(writer)
    for obj in ways:
        if obj.is_way():
            if obj.id in dropids:
                pass
            # Ugly mess that fixes (kind of) large geometries under 1 osmid that were split 
            elif obj.id in editids:
                geom = shp.geometry.shape(obj.__geo_interface__["geometry"])
                coords = [(lon, lat) for lon, lat in zip(*geom.xy)]
                refs = [nod.ref for nod in obj.nodes]
                # Is the soft check even needed? 
                softcoords = [(truncate(lon, 4), truncate(lat, 4)) for lon, lat in coords]
                # This area will be fixed later. (issue #1)
                # https://docs.osmcode.org/pyosmium/latest/cookbooks/Filtering-Data-By-Tags/#finding-backward-references-with-the-idtracker
                for _id, row in explodict.get_group(obj.id).iterrows():
                    newwaynodes = []
                    newsoftcoords = [(truncate(lon, 4), truncate(lat, 4)) for lon, lat in zip(*row.geometry.xy)]
                    softmatches = [index for index, coord in enumerate(softcoords) if coord in newsoftcoords]
                    for newnodelon, newnodelat in zip(*row.geometry.xy):
                        for i, hardlon, hardlat in [(softindex, coords[index][0], coords[index][1]) for softindex, index in enumerate(softmatches)]:
                            if (np.isclose(hardlon, newnodelon, atol=4e-6)) and (np.isclose(hardlat, newnodelat, atol=4e-6)):
                                newwaynodes.append(refs[softmatches[i]])
                            else:
                                newwaynodes.append(fakeid)
                                writer.add_node(omi.osm.mutable.Node(id=fakeid, 
                                                location=(newnodelon, newnodelat)))
                                fakeid-=1
                    # I will add relations later too.
                    writer.add_way(omi.osm.mutable.Way(
                            id=fakeid, nodes=newwaynodes,
                            tags=obj.tags))
                    fakeid-=1
    writer.close()

⭐ The pbf is created! All that's left is to plug it into [OsmAndMapCreator](https://wiki.openstreetmap.org/wiki/OsmAndMapCreator) (Takes a while and uses up a decent amount of ram)  